In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [30]:
## Read the data
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
## Preprocess the data
data = data.drop(['RowNumber','CustomerId','Surname'], axis =1)


In [32]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [33]:
# Encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [34]:
# One-hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns= onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [35]:
# Combine one hot encoded data with the dataset

data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis= 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [36]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis= 1)
y = data['EstimatedSalary']

In [37]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [38]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [39]:
##Split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2,random_state=42)


## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [40]:
## Save all pickle files

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file) 

## ANN Regression problem Statement

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [42]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # output layer for regression

])

## Compile the model
model.compile(optimizer='adam', loss= 'mean_absolute_error',metrics = ['mae'])

model.summary()

/Users/sukhdeepnarulasingh/miniconda/envs/arm-test/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir= log_dir, histogram_freq = 1)


In [44]:
# Set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [45]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs = 200,
    callbacks= [early_stopping_callback,tensorboard_callback]
)

Epoch 1/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 101342.2578 - mae: 101342.2578 - val_loss: 98685.2734 - val_mae: 98685.2734
Epoch 2/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 100200.1016 - mae: 100200.1016 - val_loss: 98435.0859 - val_mae: 98435.0859
Epoch 3/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 99780.5703 - mae: 99780.5703 - val_loss: 97934.1484 - val_mae: 97934.1484
Epoch 4/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 98646.4844 - mae: 98646.4844 - val_loss: 97221.0234 - val_mae: 97221.0234
Epoch 5/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 98419.3906 - mae: 98419.3906 - val_loss: 96342.1797 - val_mae: 96342.1797
Epoch 6/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 98515.2266 - mae: 98515.2266 - val_loss: 95365.9766 - val_mae: 95365.9766
Epoch 7/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 97110.8281 - mae: 97110.8281 - val_loss: 94404.8906 - val_mae: 94404.8906
Epoch 8/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 

In [46]:
%load_ext tensorboard



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6008 (pid 85817), started 21:18:24 ago. (Use '!kill 85817' to kill it.)

In [48]:
# Evaluate model for test data
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f"Test MAE : {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50659.7578 - mae: 50659.7578
Test MAE : 50019.546875


In [49]:
model.save('regression_model.h5')